In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
datasetpath = '/content/drive/MyDrive/bio/dataset'
databasepath = '/content/drive/MyDrive/bio/database'
utilspath = '/content/drive/MyDrive/bio/utils'
sys.path.append(datasetpath)
sys.path.append(databasepath)
sys.path.append(utilspath)

! pip3 install selenium
! pip3 install wikipedia
! pip3 install transformers
! pip3 install spacy
! pip3 install transformer
! pip3 install neuralcoref
! python -m spacy download en_core_web_md
! pip3 install bert-extractive-summarizer
! pip3 install unidecode
! pip3 install sentencepiece
! pip3 install sentence_transformers

from persons import Person, Article, DataBase, correct_str
from shortnews import ShortNews, ShortNewsItem, create_news, create_bio
import pickle, random
from random import shuffle
from utils import get_tokenizer
from tqdm.notebook import tqdm
import nltk, numpy as np
nltk.download('punkt')
import sentence_transformers


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from summarizer import Summarizer
model = Summarizer()
tokenizer = get_tokenizer('gpt2-large')

In [3]:
with open(databasepath + '/database.pickle', 'rb') as f:
  database = pickle.load(f)

# shortnews = ShortNews()
# with open(datasetpath + '/dataset.pickle', 'wb') as f:
#   pickle.dump(shortnews, f)
with open(datasetpath + '/dataset.pickle', 'rb') as f:
  shortnews = pickle.load(f)

blacklist = ['bbc.co.uk', 'bbc.com', 'Follow us', 'follow us',
             'Follow BBC', 'follow BBC', 'Follow @', 'follow @',
             'BBC Monitoring reports and analyses',
             'click here', 'download the BBC Newsbeat',
             'For iPhone go here', 'For Android go here', 'For iOS go here']

for person in tqdm(database.persons):
    print(person.names[0])

    i = shortnews.find(person.names[0])
    if i is not None:
      print('creating new bio')
      bio = person.summary
      new = shortnews.items[i].news

      bios = nltk.tokenize.sent_tokenize(correct_str(bio, first_ignore=False))
      news = nltk.tokenize.sent_tokenize(correct_str(new, first_ignore=False))
      model = sentence_transformers.SentenceTransformer(
          'paraphrase-distilroberta-base-v1')
      embeddingsb = model.encode(bios, convert_to_tensor=True)
      embeddingsn = model.encode(news, convert_to_tensor=True)

      cosine_scores = np.array(sentence_transformers.util.pytorch_cos_sim(
          embeddingsb, embeddingsn).cpu()).max(axis=1)
      cosine_scores = cosine_scores[1:].argsort()[-3:][::-1] + 1
      cosine_scores.sort()
      cosine_scores = np.insert(cosine_scores, 0, 0, axis=0)
      print(cosine_scores.tolist())

      bio = ''
      for j in cosine_scores.tolist():
        if bio != '':
          bio += ' '
        bio += bios[j]
      print(bio)

      news = new
      print(len(tokenizer.encode(news)) + len(tokenizer.encode(bio)))
      while len(tokenizer.encode(news)) + len(tokenizer.encode(bio)) > 1000:
        print('cut')
        news = news[:-200]
      print(bio, news, sep='\n')
      print(len(tokenizer.encode(news)) + len(tokenizer.encode(bio)))

      shortnews.items[i].bio = bio
      shortnews.items[i].news = news
      with open(datasetpath + '/dataset.pickle', 'wb') as f:
        pickle.dump(shortnews, f)
      continue

    news = create_news(person.articles, blacklist, model, correct_str)
    bio = create_bio(person.summary, correct_str, n_sent=3)

    print(len(tokenizer.encode(news)) + len(tokenizer.encode(bio)))
    while len(tokenizer.encode(news)) + len(tokenizer.encode(bio)) > 1000:
      print('cut')
      news = news[:-200]
    print(bio, news, sep='\n')
    print(len(tokenizer.encode(news)) + len(tokenizer.encode(bio)))

    if (len(tokenizer.encode(news)) + len(tokenizer.encode(bio)) > 200 and 
        len(tokenizer.encode(news)) + len(tokenizer.encode(bio)) <= 1000):
      shortnews.items.append(
          ShortNewsItem(name=person.names[0], news=news, bio=bio))
    else:
      print('pass')
    print()
    
    # random.Random(42).shuffle(shortnews.items)
    with open(datasetpath + '/dataset.pickle', 'wb') as f:
      pickle.dump(shortnews, f)

Melania Trump
creating new bio
[0, 1, 2, 4]
is a Slovene-American former model and businesswoman. She was the first lady of the United States from 2017 to 2021, during the presidency of her husband, Donald Trump. Trump grew up in Slovenia and worked as a fashion model through agencies in the European fashion capitals of Milan and Paris before moving to New York City in 1996. She married the real estate developer and TV personality Donald Trump in 2005 and gave birth to their son Barron in 2006.
790
is a Slovene-American former model and businesswoman. She was the first lady of the United States from 2017 to 2021, during the presidency of her husband, Donald Trump. Trump grew up in Slovenia and worked as a fashion model through agencies in the European fashion capitals of Milan and Paris before moving to New York City in 1996. She married the real estate developer and TV personality Donald Trump in 2005 and gave birth to their son Barron in 2006.
On the first night of the Republican Nat